In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from os import listdir
from os.path import isdir
from numpy import asarray, expand_dims
from sklearn.utils import shuffle

from tensorflow.keras.models import load_model
from keras_facenet import FaceNet

In [2]:
def load_face(filename):
    #carregando imagem do arquivo
    image = Image.open(filename)
    
    #converter para RGB
    image = image.convert('RGB')
    
    return asarray(image)

In [3]:
#carregando faces de um diretorio
def load_faces(directory_scr):
    
    faces = list()
    
    #iterando arquivos
    for filename in listdir(directory_scr):
        
        path = directory_scr + filename

        try:
            faces.append(load_face(path))
        except:
            print(f'Erro na imagem {path}')
        
    return faces
    

In [4]:
def load_fotos(directory_scr):
    
    X,y = list(), list()
    
    #iterando pastas por classes
    for subdir in listdir(directory_scr):
        
        path = directory_scr + '/' +subdir + '/'
        
        if not isdir(path):
            continue
        
        faces = load_faces(path)
        
        labels = [subdir for _ in range(len(faces))]
        
        #sumarizar o progresso
        print(f'>Carregadas {len(faces)} faces da classe {subdir}')
        
        X.extend(faces)
        y.extend(labels)
        
    return asarray(X), asarray(y)

In [5]:
def get_embedding(model, face_pixels):
    
    #Padronização
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean)/std
    
    #transformar a face em 1 unico exemplo
    
    # (160,160) -> (1,160,160)
    
    samples = expand_dims(face_pixels, axis=0)
    
    #Realizar a predição gerando o embedding
    yhat = model.predict(samples)
    
    return yhat[0]


In [6]:
# trainX, trainy = load_fotos(directory_scr = "/home/vitor/Documents/PROGRAMAS/Programas_Python/Condominio_mineiro/faces")
trainX, trainy = load_fotos(directory_scr = "/home/vitor/Documents/PROGRAMAS/Programas_Python/Condominio_mineiro/faces_validation")

>Carregadas 76 faces da classe glauciane
>Carregadas 72 faces da classe anino
>Carregadas 36 faces da classe steferson


In [7]:
trainy.shape

(184,)

In [8]:
trainX.shape

(184, 160, 160, 3)

In [9]:
model = load_model('facenet_keras.h5')

In [10]:
print(model.summary())

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv2d_1a_3x3 (Conv2D)         (None, 79, 79, 32)   864         ['input_1[0][0]']                
                                                                                                  
 Conv2d_1a_3x3_BatchNorm (Batch  (None, 79, 79, 32)  96          ['Conv2d_1a_3x3[0][0]']          
 Normalization)                                                                                   
                                                                                

 Block8_5_Branch_1_Conv2d_0b_1x  (None, 3, 3, 192)   110592      ['Block8_5_Branch_1_Conv2d_0a_1x1
 3 (Conv2D)                                                      _Activation[0][0]']              
                                                                                                  
 Block8_5_Branch_1_Conv2d_0b_1x  (None, 3, 3, 192)   576         ['Block8_5_Branch_1_Conv2d_0b_1x3
 3_BatchNorm (BatchNormalizatio                                  [0][0]']                         
 n)                                                                                               
                                                                                                  
 Block8_5_Branch_1_Conv2d_0b_1x  (None, 3, 3, 192)   0           ['Block8_5_Branch_1_Conv2d_0b_1x3
 3_Activation (Activation)                                       _BatchNorm[0][0]']               
                                                                                                  
 Block8_5_

In [11]:
newTrainX = list()

In [12]:
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)

In [13]:
newTrainX = asarray(newTrainX)

In [14]:
newTrainX.shape

(184, 128)

In [15]:
df = pd.DataFrame(data = newTrainX)

In [16]:
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.544852,-0.334132,1.013158,-0.264510,-0.322226,0.967415,0.504230,-0.843833,1.013700,-0.180428,...,-0.367955,-1.315426,0.205874,0.240579,0.497310,0.408430,1.272460,-0.612219,-0.497064,-1.346089
1,0.844317,-1.162868,0.905351,0.360017,-0.406176,1.203536,0.844076,-0.866428,0.714322,0.782734,...,-0.905190,-1.178861,0.944283,0.044827,0.103322,0.401164,0.749802,-0.279723,-0.492825,-1.762449
2,0.732947,-0.988929,0.490510,0.448377,-0.498199,0.797545,0.886918,-1.290659,0.748980,0.379144,...,-0.677772,-1.536889,0.909142,0.324134,0.509422,0.093849,0.504966,-0.734245,-0.515083,-1.738710
3,0.659792,-1.201293,0.433881,0.354843,-0.344341,0.769558,0.663041,-1.279091,0.752380,0.248987,...,-0.709115,-1.478215,0.655290,0.272415,0.791685,-0.094249,0.619974,-0.422794,-0.207608,-1.617176
4,0.510552,-0.453706,0.700213,-0.253367,-0.475503,0.775488,0.424801,-0.573873,1.460614,-0.154375,...,-0.402166,-1.699978,0.293937,0.420143,0.245941,0.151753,1.196004,-0.596881,-0.557328,-1.612030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-0.463136,2.031797,-0.828680,0.677253,-0.438864,-1.767040,-1.271514,-1.461169,0.322037,0.581745,...,1.543949,1.388556,1.173924,-0.699268,-1.318101,-0.913755,0.526818,0.256470,0.994789,0.812019
180,-0.553705,1.613878,-0.417959,0.387931,0.131402,-1.294715,-1.165642,-1.541113,0.309136,0.132032,...,1.731465,1.045077,1.558632,-0.642279,-1.194365,-1.038592,0.515083,0.201941,0.076190,0.950706
181,-0.447034,2.120559,-1.170795,0.405479,-0.547221,-1.574009,-1.625558,-0.939384,-0.007186,0.310999,...,1.329198,0.997773,1.265311,-0.490962,-1.330633,-0.533949,0.651528,0.539643,0.632580,0.760968
182,-0.676332,1.565943,-0.908553,0.160335,-0.093269,-1.125900,-0.745321,-1.402514,0.278515,0.107065,...,1.644190,1.363476,1.532303,0.017134,-1.364406,-0.830635,0.263987,0.800664,0.107907,0.803421


In [17]:
df['target'] = trainy

In [18]:
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,0.544852,-0.334132,1.013158,-0.264510,-0.322226,0.967415,0.504230,-0.843833,1.013700,-0.180428,...,-1.315426,0.205874,0.240579,0.497310,0.408430,1.272460,-0.612219,-0.497064,-1.346089,glauciane
1,0.844317,-1.162868,0.905351,0.360017,-0.406176,1.203536,0.844076,-0.866428,0.714322,0.782734,...,-1.178861,0.944283,0.044827,0.103322,0.401164,0.749802,-0.279723,-0.492825,-1.762449,glauciane
2,0.732947,-0.988929,0.490510,0.448377,-0.498199,0.797545,0.886918,-1.290659,0.748980,0.379144,...,-1.536889,0.909142,0.324134,0.509422,0.093849,0.504966,-0.734245,-0.515083,-1.738710,glauciane
3,0.659792,-1.201293,0.433881,0.354843,-0.344341,0.769558,0.663041,-1.279091,0.752380,0.248987,...,-1.478215,0.655290,0.272415,0.791685,-0.094249,0.619974,-0.422794,-0.207608,-1.617176,glauciane
4,0.510552,-0.453706,0.700213,-0.253367,-0.475503,0.775488,0.424801,-0.573873,1.460614,-0.154375,...,-1.699978,0.293937,0.420143,0.245941,0.151753,1.196004,-0.596881,-0.557328,-1.612030,glauciane
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-0.463136,2.031797,-0.828680,0.677253,-0.438864,-1.767040,-1.271514,-1.461169,0.322037,0.581745,...,1.388556,1.173924,-0.699268,-1.318101,-0.913755,0.526818,0.256470,0.994789,0.812019,steferson
180,-0.553705,1.613878,-0.417959,0.387931,0.131402,-1.294715,-1.165642,-1.541113,0.309136,0.132032,...,1.045077,1.558632,-0.642279,-1.194365,-1.038592,0.515083,0.201941,0.076190,0.950706,steferson
181,-0.447034,2.120559,-1.170795,0.405479,-0.547221,-1.574009,-1.625558,-0.939384,-0.007186,0.310999,...,0.997773,1.265311,-0.490962,-1.330633,-0.533949,0.651528,0.539643,0.632580,0.760968,steferson
182,-0.676332,1.565943,-0.908553,0.160335,-0.093269,-1.125900,-0.745321,-1.402514,0.278515,0.107065,...,1.363476,1.532303,0.017134,-1.364406,-0.830635,0.263987,0.800664,0.107907,0.803421,steferson


In [19]:
# df.to_csv('faces.csv')
df.to_csv('faces_validation.csv')

In [20]:
X,y = shuffle(newTrainX, trainy, random_state = 0)